In [28]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from string import punctuation

In [29]:
df = pd.read_csv('data/ProductsDataset.csv')
df.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35548 entries, 0 to 35547
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           35548 non-null  object 
 1   descrirption    33537 non-null  object 
 2   product_id      35536 non-null  object 
 3   category_id     35536 non-null  float64
 4   subcategory_id  35536 non-null  object 
 5   properties      35536 non-null  object 
 6   image_links     35533 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.9+ MB


In [31]:
df.isna().sum()

title                0
descrirption      2011
product_id          12
category_id         12
subcategory_id      12
properties          12
image_links         15
dtype: int64

In [32]:
df.fillna(0, inplace=True)

#### Осуществлён препроцессинг текста (как минимум удаление знаков препинания, приведение к нижнему регистру, стемминг/лемматизация).

In [33]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

'''def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)
'''

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [34]:
df['descrirption'] = df['descrirption'].apply(preprocess_txt)
df['title'] = df['title'].apply(preprocess_txt)
df = df.drop('image_links', axis=1)

In [35]:
df.head()

,title,descrirption,product_id,category_id,subcategory_id,properties
0,юбка детский orby,новый не носить ни раз в реал красивый чем на ...,58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}"
1,ботильон,новыепривезти из чехия указать размер 40ный ма...,5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota..."
2,брюки,размер 4042 брюки почти новый не знать как мер...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...
3,продать детский шапка,продать шапкикажда 200ррозовый и белый проданна,57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_..."
4,блузка,темносиний 42 размерсостояние отличноекак новы...,5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay..."


#### Текст векторизирован любым из способов (CountVectorizer, TfidfVectorizer, HashingVectorizer, Word2Vec).

In [13]:
X = df.drop('product_id',axis=1)
Y = df['product_id']
train_X, test_X, train_y, test_y = train_test_split(X,Y)

In [14]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit(train_X)

In [15]:
X_train = X.transform(train_X)
y_train = np.array(train_y)

X_test = X.transform(test_X)
y_test = np.array(test_y)

In [16]:
model = LogisticRegression()
model.fit(X_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [6, 26661]